# PreProcessing

Now that we have the base layers organized into a base_layers group in the contents frame, we'll use python to iterate through each layer, preparing them to be used in the calculation of the walkscore in the next notebook.

### Library Imports

In [1]:
import arcpy
import arcpy.mp
import pandas as pd
import geopandas as gpd
import os
import numpy as np
from arcpy.sa import Int

from collections import defaultdict

### Step 0: Formatting Fishnet

In [2]:
fishnet_layer = r"C:\Users\rtvpd\Documents\Walkability_Seattle\Walkability_Seattle.gdb\fishnet_clipped"
base_layers_group = r"C:\Users\rtvpd\Documents\Walkability_Seattle\Walkability_Seattle.gdb"
output_gdb = r"C:\Users\rtvpd\Documents\Walkability_Seattle\Walkability_Seattle.gdb"
geodatabase_path = r"C:\Users\rtvpd\Documents\Walkability_Seattle\Walkability_Seattle.gdb"
fishnet_clipped = r"C:\Users\rtvpd\Documents\Walkability_Seattle\Walkability_Seattle.gdb\fishnet_clipped"

arcpy.env.workspace = r"C:\Users\rtvpd\Documents\Walkability_Seattle\Walkability_Seattle.gdb"
fishnet_area_field = "total_area"

In [3]:
output_folder = r"C:\Users\rtvpd\Documents\Walkability_Seattle\output"
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

In [4]:
fields = arcpy.ListFields(fishnet_layer)
for field in fields:
    print(f"Field Name: {field.name}, Field Type: {field.type}")

Field Name: OBJECTID, Field Type: OID
Field Name: Shape, Field Type: Geometry
Field Name: Shape_Length, Field Type: Double
Field Name: Shape_Area, Field Type: Double
Field Name: IndexID, Field Type: Integer


In [5]:
# Add the IndexID field if it doesn't exist
index_field = "IndexID"
if not any(f.name == index_field for f in arcpy.ListFields(fishnet_clipped)):
    arcpy.management.AddField(fishnet_clipped, index_field, "LONG")

# Populate the IndexID field with unique values
with arcpy.da.UpdateCursor(fishnet_clipped, [index_field]) as cursor:
    for i, row in enumerate(cursor):
        row[0] = i + 1
        cursor.updateRow(row)

print("IndexID field created and populated in fishnet_clipped.")

IndexID field created and populated in fishnet_clipped.


### Step 1: Setup

#### Mandatory Layers

In [6]:
base_layers = [
#     "TreeCanopy",
#     "Public_Amenities",
    "Business_Amenities",
    "Industrial",
    "ParkingLots",
    "GolfCourse",
    "Cemeteries",
    "Hospitals",
    "Slope",
    "Bike_greenways",
    "Bike_protected",
    "Bike_buffer",
    "Healthy_Streets",
    "Parks",
    "Universities",
    "Sidewalks",
    "Plaza",
    "trails",
    "MultiUseTrails",
    "Streets",
    "population",
    "crashes",
    "SPD_Crime_Data"
]

In [7]:
for layer_name in base_layers:
    input_layer = f"{base_layers_group}\\{layer_name}"
    print(f"Checking for layer: {input_layer}")  # Add this line
    if not arcpy.Exists(input_layer):
        print(f"Layer {input_layer} does not exist. Skipping.")
        continue

Checking for layer: C:\Users\rtvpd\Documents\Walkability_Seattle\Walkability_Seattle.gdb\Business_Amenities
Checking for layer: C:\Users\rtvpd\Documents\Walkability_Seattle\Walkability_Seattle.gdb\Industrial
Checking for layer: C:\Users\rtvpd\Documents\Walkability_Seattle\Walkability_Seattle.gdb\ParkingLots
Checking for layer: C:\Users\rtvpd\Documents\Walkability_Seattle\Walkability_Seattle.gdb\GolfCourse
Checking for layer: C:\Users\rtvpd\Documents\Walkability_Seattle\Walkability_Seattle.gdb\Cemeteries
Checking for layer: C:\Users\rtvpd\Documents\Walkability_Seattle\Walkability_Seattle.gdb\Hospitals
Checking for layer: C:\Users\rtvpd\Documents\Walkability_Seattle\Walkability_Seattle.gdb\Slope
Checking for layer: C:\Users\rtvpd\Documents\Walkability_Seattle\Walkability_Seattle.gdb\Bike_greenways
Checking for layer: C:\Users\rtvpd\Documents\Walkability_Seattle\Walkability_Seattle.gdb\Bike_protected
Checking for layer: C:\Users\rtvpd\Documents\Walkability_Seattle\Walkability_Seattle.gdb\

#### Functions

In [8]:
def calculate_average_slope(fishnet_layer, slope_raster, features_layer, output_table):
    arcpy.CheckOutExtension("Spatial")
    extracted_slope = arcpy.sa.ExtractByMask(slope_raster, features_layer)
    temp_extracted_slope = f"{output_gdb}\\temp_extracted_slope"
    extracted_slope.save(temp_extracted_slope)
    arcpy.sa.ZonalStatisticsAsTable(fishnet_layer, "IndexID", temp_extracted_slope, output_table, "NODATA", "MEAN")
    arcpy.management.Delete(temp_extracted_slope)

def integrate_slope_and_area(intersect_output, slope_output_table, area_field, effective_slope_field):
    arcpy.management.AddField(intersect_output, effective_slope_field, "DOUBLE")
    slope_df = arcpy.da.TableToNumPyArray(slope_output_table, ["IndexID", "MEAN"])
    slope_df = pd.DataFrame(slope_df)
    with arcpy.da.UpdateCursor(intersect_output, ["IndexID", area_field, effective_slope_field]) as cursor:
        for row in cursor:
            mean_slope = slope_df.loc[slope_df["IndexID"] == row[0], "MEAN"]
            row[2] = (mean_slope.values[0] * row[1]) if not mean_slope.empty and row[1] is not None else 0
            cursor.updateRow(row)

def calculate_polygon_area(layer, area_field):
    if not any(f.name.lower() == area_field.lower() for f in arcpy.ListFields(layer)):
        arcpy.management.AddField(layer, area_field, "DOUBLE")
    arcpy.management.CalculateGeometryAttributes(layer, [[area_field, "AREA_GEODESIC"]], area_unit="SQUARE_FEET_US")

def calculate_polyline_area_with_recalculated_length(layer, area_field, width_field):
    recalculated_length_field = f"{area_field}_len"
    if not any(f.name.lower() == recalculated_length_field.lower() for f in arcpy.ListFields(layer)):
        arcpy.management.AddField(layer, recalculated_length_field, "DOUBLE")
    arcpy.management.CalculateGeometryAttributes(layer, [[recalculated_length_field, "LENGTH_GEODESIC"]], length_unit="FEET_US")
    if not any(f.name.lower() == area_field.lower() for f in arcpy.ListFields(layer)):
        arcpy.management.AddField(layer, area_field, "DOUBLE")
    with arcpy.da.UpdateCursor(layer, [width_field, recalculated_length_field, area_field]) as cursor:
        for row in cursor:
            row[2] = row[0] * row[1] if row[0] is not None and row[1] is not None else 0
            cursor.updateRow(row)

def calculate_effective_area(layer, effective_area_field, length_field="Shape_Length", width_field="street_width", speed_limit_field="SPEEDLIMIT", at_grade_field="at_grade_scalar"):
    recalculated_length_field = f"{effective_area_field}_len"
    if not any(f.name.lower() == recalculated_length_field.lower() for f in arcpy.ListFields(layer)):
        arcpy.management.AddField(layer, recalculated_length_field, "DOUBLE")
    arcpy.management.CalculateGeometryAttributes(layer, [[recalculated_length_field, "LENGTH_GEODESIC"]], length_unit="FEET_US")

    if not any(f.name == effective_area_field for f in arcpy.ListFields(layer)):
        arcpy.management.AddField(layer, effective_area_field, "DOUBLE")

    with arcpy.da.UpdateCursor(layer, [recalculated_length_field, width_field, speed_limit_field, at_grade_field, effective_area_field]) as cursor:
        for row in cursor:
            if row[0] is not None and row[1] is not None and row[2] is not None and row[3] is not None:
                row[4] = row[0] * row[1] * row[2] * row[3]
            else:
                row[4] = None
            cursor.updateRow(row)
    print(f"Calculated effective area for {layer} and stored in {effective_area_field}.")
    
def create_layer(input_layer, fclass_list, output_layer_name):
    # Create a query to filter the input layer based on fclass values
    fclass_query = f"""fclass IN ({','.join([f"'{fc}'" for fc in fclass_list])})"""
    
    # Create the output layer
    arcpy.management.MakeFeatureLayer(input_layer, "temp_layer", fclass_query)
    output_layer = f"{workspace}\\{output_layer_name}"
    
    # Check if the output layer already exists and delete it if it does
    if arcpy.Exists(output_layer):
        arcpy.management.Delete(output_layer)
    
    # Save the filtered features to a new feature class
    arcpy.management.CopyFeatures("temp_layer", output_layer)
    print(f"Created {output_layer_name} layer with {len(fclass_list)} fclass values.")
    
def calculate_counts(input_layer, intersect_output, fishnet_layer, summary_output, id_field):
    # Intersect the input crime data layer with the fishnet
    arcpy.analysis.Intersect([input_layer, fishnet_layer], intersect_output)

    # Add IndexID to intersect output if it doesn't exist
    if not any(f.name == "IndexID" for f in arcpy.ListFields(intersect_output)):
        arcpy.management.AddField(intersect_output, "IndexID", "LONG")
        # Populate the IndexID field in intersect output
        with arcpy.da.UpdateCursor(intersect_output, ["IndexID"]) as cursor:
            for i, row in enumerate(cursor):
                row[0] = i + 1
                cursor.updateRow(row)

    # Calculate the count of crimes within each fishnet grid cell
    arcpy.analysis.Statistics(intersect_output, summary_output, [[id_field, "COUNT"]], "IndexID")

    # Join the summary table back to the fishnet layer
    count_field = f"COUNT_{id_field}"
    arcpy.management.JoinField(fishnet_layer, "IndexID", summary_output, "IndexID", [count_field])

    # Update null values in the joined count field to 0
    with arcpy.da.UpdateCursor(fishnet_layer, [count_field]) as cursor:
        for row in cursor:
            if row[0] is None:
                row[0] = 0  # Set null counts to 0
            cursor.updateRow(row)

    print(f"Joined {count_field} to {fishnet_layer} and created summary table {summary_output}.")
    
def calculate_crime_density(input_layer, intersect_output, fishnet_layer, summary_output, id_field):
    # Intersect the input crime data layer with the fishnet
    arcpy.analysis.Intersect([input_layer, fishnet_layer], intersect_output)

    # Add IndexID to intersect output if it doesn't exist
    if not any(f.name == "IndexID" for f in arcpy.ListFields(intersect_output)):
        arcpy.management.AddField(intersect_output, "IndexID", "LONG")
        # Populate the IndexID field in intersect output
        with arcpy.da.UpdateCursor(intersect_output, ["IndexID"]) as cursor:
            for i, row in enumerate(cursor):
                row[0] = i + 1
                cursor.updateRow(row)

    # Calculate the count of crimes within each fishnet grid cell
    arcpy.analysis.Statistics(intersect_output, summary_output, [[id_field, "COUNT"]], "IndexID")

    # Join the summary table back to the fishnet layer
    count_field = f"COUNT_{id_field}"
    arcpy.management.JoinField(fishnet_layer, "IndexID", summary_output, "IndexID", [count_field])

    # Update null values in the joined count field to 0
    with arcpy.da.UpdateCursor(fishnet_layer, [count_field]) as cursor:
        for row in cursor:
            if row[0] is None:
                row[0] = 0  # Set null counts to 0
            cursor.updateRow(row)

    print(f"Joined {count_field} to {fishnet_layer} and created summary table {summary_output}.")
    
def calculate_population_density(input_layer, intersect_output, fishnet_layer, summary_output, id_field, density_field):
    # Intersect the input population layer with the fishnet
    arcpy.analysis.Intersect([input_layer, fishnet_layer], intersect_output)

    # Add IndexID to intersect output if it doesn't exist
    if not any(f.name == "IndexID" for f in arcpy.ListFields(intersect_output)):
        arcpy.management.AddField(intersect_output, "IndexID", "LONG")
        # Populate the IndexID field in intersect output
        with arcpy.da.UpdateCursor(intersect_output, ["IndexID"]) as cursor:
            for i, row in enumerate(cursor):
                row[0] = i + 1
                cursor.updateRow(row)

    # Calculate the area of each intersected polygon (geodesic area in ft²)
    area_field = "intersect_area"
    if not any(f.name == area_field for f in arcpy.ListFields(intersect_output)):
        arcpy.management.AddField(intersect_output, area_field, "DOUBLE")

        try:
            # Try calculating geometry attributes with geodesic area
            arcpy.management.CalculateGeometryAttributes(intersect_output, [[area_field, "AREA_GEODESIC"]], area_unit="SQUARE_FEET_US")
        except arcpy.ExecuteError:
            # Fallback: Use Add Geometry Attributes tool
            arcpy.management.AddGeometryAttributes(intersect_output, "AREA_GEODESIC", Area_Unit="SQUARE_FEET_US")
            arcpy.management.CalculateField(intersect_output, area_field, "!POLY_AREA!", "PYTHON3")

    # Calculate the proportional population for each intersected area
    proportional_population_field = "proportional_population"
    if not any(f.name == proportional_population_field for f in arcpy.ListFields(intersect_output)):
        arcpy.management.AddField(intersect_output, proportional_population_field, "DOUBLE")

    with arcpy.da.UpdateCursor(intersect_output, [area_field, density_field, proportional_population_field]) as cursor:
        for row in cursor:
            if row[0] is not None and row[1] is not None:
                row[2] = row[0] * row[1]  # proportional_population = intersect_area * population_density
            else:
                row[2] = 0
            cursor.updateRow(row)

    # Summarize the proportional population by fishnet grid (using IndexID)
    arcpy.analysis.Statistics(intersect_output, summary_output, [[proportional_population_field, "SUM"]], "IndexID")

    # Determine the correct name of the output field from summary statistics
    summary_field_name = f"SUM_{proportional_population_field}"

    # Join the summary table back to the fishnet layer
    arcpy.management.JoinField(fishnet_layer, "IndexID", summary_output, "IndexID", [summary_field_name])

    # Update null values in the joined population field to 0
    with arcpy.da.UpdateCursor(fishnet_layer, [summary_field_name]) as cursor:
        for row in cursor:
            if row[0] is None:
                row[0] = 0  # Set null population to 0
            cursor.updateRow(row)

    print(f"Joined {summary_field_name} to {fishnet_layer} and created summary table {summary_output}.")


def calculate_max_speed_limit(intersect_layer, fishnet_layer, output_table, speed_limit_field):
    # Calculate the max speed limit for each intersected grid cell
    arcpy.analysis.Statistics(intersect_layer, output_table, [[speed_limit_field, "MAX"]], "IndexID")
    # Join the result back to the fishnet layer
    arcpy.management.JoinField(fishnet_layer, "IndexID", output_table, "IndexID", ["MAX_" + speed_limit_field])
    # Rename the field to Max_Speed_Limit
    arcpy.management.AlterField(fishnet_layer, "MAX_" + speed_limit_field, "Max_Speed_Limit")
    
def calculate_average_density(fishnet_layer, density_raster, features_layer, output_table):
    """Calculate average density for each fishnet grid and save as a table."""
    arcpy.CheckOutExtension("Spatial")
    extracted_density = arcpy.sa.ExtractByMask(density_raster, features_layer)
    temp_extracted_density = f"{output_gdb}\\temp_extracted_density"
    extracted_density.save(temp_extracted_density)
    arcpy.sa.ZonalStatisticsAsTable(fishnet_layer, "IndexID", temp_extracted_density, output_table, "NODATA", "MEAN")
    arcpy.management.Delete(temp_extracted_density)
    print(f"Average density calculated and saved to {output_table}.")

#### Calculating the Effective Slope

Since the slope data is provided in a raster, I'll need to segment this data to use only the slope data pertinent to the layers in my dataset. In the below function, we'll take the raster data and mask it with the layers provided in comb_feats, a list of features that we'll use to calculate the average slope.

Using these combined features we can determine the exact slope of the sidewalk in a fishnet grid, rather than use the average slope over a grid as a proxy for the slope of the infrastructure a person will actually be using.

In [9]:
arcpy.env.overwriteOutput = True  # Allow outputs to be overwritten

# Get the spatial reference of the fishnet layer
fishnet_sr = arcpy.Describe(fishnet_layer).spatialReference

# Define the walkscore_fishnet_layer
walkscore_fishnet_layer = f"{output_gdb}\\walkscore_fishnet"

# Check if the walkscore_fishnet_layer exists and delete it if it does
if arcpy.Exists(walkscore_fishnet_layer):
    arcpy.management.Delete(walkscore_fishnet_layer)
    print(f"Deleted existing {walkscore_fishnet_layer}.")

# Create a copy of the fishnet layer to work on
arcpy.management.CopyFeatures(fishnet_layer, walkscore_fishnet_layer)
print("Copied fishnet_clipped to walkscore_fishnet.")

# Add a new field for indexing and populate it with unique values
index_field = "IndexID"
if not any(f.name == index_field for f in arcpy.ListFields(walkscore_fishnet_layer)):
    arcpy.management.AddField(walkscore_fishnet_layer, index_field, "LONG")

# Populate the new index field with unique values
with arcpy.da.UpdateCursor(walkscore_fishnet_layer, [index_field]) as cursor:
    for i, row in enumerate(cursor):
        row[0] = i + 1
        cursor.updateRow(row)

print("Index field populated with unique values.")

# Add a new field for total area if it doesn't exist
total_area_field = "total_area"
if not any(f.name == total_area_field for f in arcpy.ListFields(walkscore_fishnet_layer)):
    arcpy.management.AddField(walkscore_fishnet_layer, total_area_field, "DOUBLE")

# Calculate the total area for each fishnet grid cell
arcpy.management.CalculateGeometryAttributes(walkscore_fishnet_layer, [[total_area_field, "AREA_GEODESIC"]])
print("Calculated total area for each fishnet grid cell.")

Deleted existing C:\Users\rtvpd\Documents\Walkability_Seattle\Walkability_Seattle.gdb\walkscore_fishnet.
Copied fishnet_clipped to walkscore_fishnet.
Index field populated with unique values.
Calculated total area for each fishnet grid cell.


In [10]:
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

### Step 2: Process Each Layer and Calculate Allocations for each Fishnet Grid

#### Effective Area Scalers

In [11]:
# norm_fields = [sidewalk_score_field, park_score_field, trail_score_field, street_score_field, bike_score_field]

In [12]:
scalers = {
    "parkinglots": 1,
    "industrial": 1,
    "golfcourse": 1,
    "hospitals": 1,
    "cemeteries": 1
}

#### Processing Public Amenity Data & Separating Out Amenity Types

In [13]:
workspace = r"C:\Users\rtvpd\Documents\Walkability_Seattle\Walkability_Seattle.gdb"
layer = r"C:\Users\rtvpd\Documents\Walkability_Seattle\Walkability_Seattle.gdb\PointsofInterest"

# Use a set to collect unique fclass values
fclass_set = set()

# Use a SearchCursor to iterate through the fclass field
with arcpy.da.SearchCursor(layer, ["fclass"]) as cursor:
    for row in cursor:
        fclass_set.add(row[0])

# Print the unique fclass values and their count
unique_fclass_count = len(fclass_set)
print(f"Number of unique fclass values: {unique_fclass_count}")
print("Unique fclass values:")
for fclass in fclass_set:
    print(fclass)

Number of unique fclass values: 108
Unique fclass values:
supermarket
computer_shop
library
bank
vending_any
bookshop
college
recycling_clothes
pitch
community_centre
laundry
sports_centre
video_shop
arts_centre
newsagent
drinking_water
recycling_glass
fast_food
university
biergarten
doityourself
guesthouse
tower
attraction
picnic_site
greengrocer
pharmacy
furniture_shop
jeweller
school
playground
swimming_pool
clothes
embassy
theatre
recycling
post_office
artwork
veterinary
doctors
travel_agent
florist
vending_machine
dentist
museum
toilet
bench
garden_centre
pub
beverages
restaurant
water_well
bar
hospital
comms_tower
vending_parking
sports_shop
optician
bakery
car_dealership
car_sharing
toy_shop
cafe
dog_park
nightclub
bicycle_shop
car_rental
recycling_paper
clinic
mall
hostel
gift_shop
monument
courthouse
shelter
bicycle_rental
atm
stationery
convenience
fire_station
wastewater_plant
butcher
chemist
hairdresser
camera_surveillance
food_court
beauty_shop
general
kindergarten
mobile_

In [14]:
business_amenities = [
    'supermarket', 'convenience', 'greengrocer', 'butcher', 'department_store', 'mall', 
    'gift_shop', 'shoe_shop', 'clothes', 'bookshop', 'stationery', 'furniture_shop', 'jeweller', 
    'computer_shop', 'mobile_phone_shop', 'outdoor_shop', 'general', 'florist', 'toy_shop',
    'beauty_shop', 'laundry', 'bank', 'atm', 'cafe', 'restaurant', 
    'pub', 'bar', 'fast_food', 'bakery', 'food_court', 'beverages', 'nightclub', 'car_sharing',
    'car_wash', 'video_shop', 'vending_any', 'theatre', 'museum', 'attraction', 'cinema',
    'market_place', 'mobile_phone_shop', 'bookshop', 'laundry', 'mobile_phone_shop',
    'garden_centre','doityourself','hairdresser','bicycle_shop','biergarten','sports_shop'
]
public_amenities = [
    'bench', 'drinking_water', 'waste_basket', 'library', 'post_box','post_office', 'recycling', 
    'recycling_glass', 'recycling_paper', 'vending_machine', 'artwork', 'tourist_info',
    'viewpoint', 'monument', 'picnic_site', 'memorial', 'fountain', 'shelter', 'public_building',
    'arts_centre','courthouse','community_centre'
]

#### Main Processing Block

In [15]:
# Main processing loop for preprocessing layers
created_layers = []

for layer_name in base_layers:
    print(f"Processing layer: {layer_name}")

    input_layer = f"{base_layers_group}\\{layer_name}"

    if not arcpy.Exists(input_layer):
        print(f"Layer {input_layer} does not exist. Skipping.")
        continue

    desc = arcpy.Describe(input_layer)
    if hasattr(desc, "shapeType"):
        geometry_type = desc.shapeType
    else:
        print(f"Layer {layer_name} does not have a shapeType attribute. Skipping.")
        continue

    # Handle crash layer
    if layer_name.lower() == "crashes":
        intersect_output = f"{workspace}\\{layer_name}_intersect"
        summary_output = f"{workspace}\\{layer_name}_sum"
        id_field = "OBJECTID"
        calculate_counts(input_layer, intersect_output, walkscore_fishnet_layer, summary_output, id_field)
        created_layers.append(summary_output)
        continue

    # Handle Population layer
    if layer_name.lower() == "population":
        intersect_output = f"{workspace}\\{layer_name}_intersect"
        summary_output = f"{workspace}\\{layer_name}_sum"
        id_field = "OBJECTID"
        density_field = "density_2023"
        calculate_population_density(input_layer, intersect_output, walkscore_fishnet_layer, summary_output, id_field, density_field)
        created_layers.append(summary_output)
        continue
        
    # Handle crash layer
    if layer_name.lower() == "SPD_Crime_Data":
        intersect_output = f"{workspace}\\{layer_name}_intersect"
        summary_output = f"{workspace}\\{layer_name}_sum"
        id_field = "OBJECTID"
        calculate_counts(input_layer, intersect_output, walkscore_fishnet_layer, summary_output, id_field)
        created_layers.append(summary_output)
        continue

    # Skip Point layers (e.g., business_amenities) for now; handle them later for Kernel Density
    if geometry_type == "Point" and layer_name.endswith("_Amenities"):
        print(f"Skipping point layer {layer_name} for separate kernel density processing.")
        continue

    # Handle Polygons and Polylines as before
    input_layer_sr = desc.spatialReference
    fishnet_sr = arcpy.Describe(walkscore_fishnet_layer).spatialReference
    projected_layer = f"{workspace}\\{layer_name}_proj"

    if input_layer_sr.name != fishnet_sr.name:
        if arcpy.Exists(projected_layer):
            arcpy.management.Delete(projected_layer)
        arcpy.management.Project(input_layer, projected_layer, fishnet_sr)
    else:
        projected_layer = input_layer

    intersect_output = f"{workspace}\\{layer_name}_int"

    if arcpy.Exists(intersect_output):
        arcpy.management.Delete(intersect_output)

    arcpy.analysis.Intersect([walkscore_fishnet_layer, projected_layer], intersect_output)

    if layer_name.lower() == "streets":
        effective_area_field = f"{layer_name}_effective_area"
        calculate_effective_area(intersect_output, effective_area_field)
        area_field = effective_area_field

        # Run the calculate_max_speed_limit function here for the Streets layer
        output_table = f"{output_gdb}\\max_speed_limit_Streets_int"
        calculate_max_speed_limit(intersect_output, walkscore_fishnet_layer, output_table, "effective_SPEEDLIMIT")

    elif layer_name.lower() in scalers.keys():
        area_field = f"{layer_name}_area"
        area_field = area_field.replace("-", "_").replace(" ", "_")
        if geometry_type == "Polygon":
            calculate_polygon_area(intersect_output, area_field)
        effective_area_field = f"{layer_name}_effective_area"
        if not any(f.name == effective_area_field for f in arcpy.ListFields(intersect_output)):
            arcpy.management.AddField(intersect_output, effective_area_field, "DOUBLE")
        scaler = scalers[layer_name.lower()]
        with arcpy.da.UpdateCursor(intersect_output, [area_field, effective_area_field]) as cursor:
            for row in cursor:
                if row[0] is not None:
                    row[1] = row[0] * scaler
                else:
                    row[1] = None
                cursor.updateRow(row)
        area_field = effective_area_field
    else:
        area_field = f"{layer_name}_area"
        area_field = area_field.replace("-", "_").replace(" ", "_")
        if geometry_type == "Polygon":
            calculate_polygon_area(intersect_output, area_field)
        elif geometry_type == "Polyline":
            width_field = None
            for field in arcpy.ListFields(intersect_output):
                if field.name.lower().endswith("width"):
                    width_field = field.name
            if width_field:
                calculate_polyline_area_with_recalculated_length(intersect_output, area_field, width_field)
            else:
                print(f"Width field not found for {layer_name}, skipping area calculation.")

    if not any(f.name.lower() == area_field.lower() for f in arcpy.ListFields(intersect_output)):
        print(f"Area field {area_field} was not created for {layer_name}, skipping summary statistics.")
        continue

    summary_output = f"{workspace}\\{layer_name}_sum"
    if arcpy.Exists(summary_output):
        arcpy.management.Delete(summary_output)

    if not any(f.name == index_field for f in arcpy.ListFields(intersect_output)):
        arcpy.management.AddField(intersect_output, index_field, "LONG")
        with arcpy.da.UpdateCursor(intersect_output, [index_field]) as cursor:
            for i, row in enumerate(cursor):
                row[0] = i + 1
                cursor.updateRow(row)

    arcpy.analysis.Statistics(intersect_output, summary_output, [[area_field, "SUM"]], index_field)
    created_layers.append(summary_output)

print("Main processing complete for all layers.")

Processing layer: Business_Amenities
Skipping point layer Business_Amenities for separate kernel density processing.
Processing layer: Industrial
Processing layer: ParkingLots
Processing layer: GolfCourse
Processing layer: Cemeteries
Processing layer: Hospitals
Processing layer: Slope
Layer Slope does not have a shapeType attribute. Skipping.
Processing layer: Bike_greenways
Processing layer: Bike_protected
Processing layer: Bike_buffer
Processing layer: Healthy_Streets
Processing layer: Parks
Processing layer: Universities
Processing layer: Sidewalks
Processing layer: Plaza
Processing layer: trails
Processing layer: MultiUseTrails
Processing layer: Streets
Calculated effective area for C:\Users\rtvpd\Documents\Walkability_Seattle\Walkability_Seattle.gdb\Streets_int and stored in Streets_effective_area.
Processing layer: population
Joined SUM_proportional_population to C:\Users\rtvpd\Documents\Walkability_Seattle\Walkability_Seattle.gdb\walkscore_fishnet and created summary table C:\Us

#### Slope Processing Loop

In [16]:
slope_layers = ['Sidewalks', 'Streets', 'MultiUseTrails', 'trails']

# Calculate slope for entire grid
grid_slope_output_table = f"{output_gdb}\\grid_slope"
calculate_average_slope(walkscore_fishnet_layer, "Slope", walkscore_fishnet_layer, grid_slope_output_table)

# Check if "Grid_Slope_MEAN" field already exists, and join or alter as necessary
if not any(f.name == "Grid_Slope_MEAN" for f in arcpy.ListFields(walkscore_fishnet_layer)):
    arcpy.management.JoinField(walkscore_fishnet_layer, "IndexID", grid_slope_output_table, "IndexID", "MEAN")
    arcpy.management.AlterField(walkscore_fishnet_layer, "MEAN", "Grid_Slope_MEAN")
else:
    print("Grid_Slope_MEAN field already exists. Skipping join and alter operations.")

# Process slope for specific polyline layers
for layer_name in slope_layers:
    print(f"Processing slope for layer: {layer_name}")

    input_layer = f"{base_layers_group}\\{layer_name}"
    
    if not arcpy.Exists(input_layer):
        print(f"Layer {input_layer} does not exist. Skipping.")
        continue

    desc = arcpy.Describe(input_layer)
    if hasattr(desc, "shapeType"):
        geometry_type = desc.shapeType
        if geometry_type != "Polyline":
            print(f"Layer {layer_name} is not a polyline. Skipping slope calculation.")
            continue
    else:
        print(f"Layer {layer_name} does not have a shapeType attribute. Skipping.")
        continue

    intersect_output = f"{workspace}\\{layer_name}_int"
    slope_output_table = f"{workspace}\\{layer_name}_slope"

    if arcpy.Exists(intersect_output):
        print(f"Calculating slope for {layer_name}")
        calculate_average_slope(intersect_output, "Slope", intersect_output, slope_output_table)

        effective_slope_field = f"{layer_name}_Slope_Mean"

        # Ensure the effective_slope_field exists
        if not any(f.name == effective_slope_field for f in arcpy.ListFields(walkscore_fishnet_layer)):
            arcpy.management.AddField(walkscore_fishnet_layer, effective_slope_field, "DOUBLE")

        slope_df = arcpy.da.TableToNumPyArray(slope_output_table, ["IndexID", "MEAN"])
        slope_dict = {row["IndexID"]: row["MEAN"] for row in slope_df}

        with arcpy.da.UpdateCursor(walkscore_fishnet_layer, ["IndexID", effective_slope_field]) as cursor:
            for row in cursor:
                row[1] = slope_dict.get(row[0], None)
                cursor.updateRow(row)
    else:
        print(f"Intersect output for {layer_name} does not exist. Skipping.")

print("Slope processing complete for all layers.")

# Ensure that the combined slope mean is calculated correctly
slope_fields = [f"{layer}_Slope_Mean" for layer in slope_layers if any(f.name == f"{layer}_Slope_Mean" for f in arcpy.ListFields(walkscore_fishnet_layer))]

# Collect all necessary data in one go
all_data = {
    row[0]: {
        field: row[idx + 1] for idx, field in enumerate(slope_fields + ["Grid_Slope_MEAN"])
    } for row in arcpy.da.SearchCursor(walkscore_fishnet_layer, ["IndexID"] + slope_fields + ["Grid_Slope_MEAN"])
}

# Debug: Print a few entries to check data integrity
print("Sample data from all_data for debugging:")
for idx, (key, value) in enumerate(all_data.items()):
    if idx < 5:  # Print first 5 entries
        print(f"IndexID: {key}, Data: {value}")
    else:
        break

# Update the effective_slope field based on the collected data
if not any(f.name == "effective_slope" for f in arcpy.ListFields(walkscore_fishnet_layer)):
    arcpy.management.AddField(walkscore_fishnet_layer, "effective_slope", "DOUBLE")

with arcpy.da.UpdateCursor(walkscore_fishnet_layer, ["IndexID", "effective_slope"] + slope_fields + ["Grid_Slope_MEAN"]) as cursor:
    for row in cursor:
        index_id = row[0]
        sidewalks_slope = all_data[index_id].get("Sidewalks_Slope_Mean")
        multiuse_trails_slope = all_data[index_id].get("MultiUseTrails_Slope_Mean")
        streets_slope = all_data[index_id].get("Streets_Slope_Mean")
        grid_slope_mean = all_data[index_id]["Grid_Slope_MEAN"]

        if sidewalks_slope is not None:
            slope_mean = sidewalks_slope
        elif multiuse_trails_slope is not None:
            slope_mean = multiuse_trails_slope
        elif streets_slope is not None:
            slope_mean = streets_slope
        else:
            slope_mean = grid_slope_mean

        row[1] = slope_mean
        cursor.updateRow(row)

print("Combined slope mean calculated and updated.")

Processing slope for layer: Sidewalks
Calculating slope for Sidewalks
Processing slope for layer: Streets
Calculating slope for Streets
Processing slope for layer: MultiUseTrails
Calculating slope for MultiUseTrails
Processing slope for layer: trails
Calculating slope for trails
Slope processing complete for all layers.
Sample data from all_data for debugging:
IndexID: 1, Data: {'Sidewalks_Slope_Mean': 2.6028627527171175, 'Streets_Slope_Mean': 2.4719635209729587, 'MultiUseTrails_Slope_Mean': None, 'trails_Slope_Mean': None, 'Grid_Slope_MEAN': 3.0777811209360757}
IndexID: 2, Data: {'Sidewalks_Slope_Mean': 1.8433810224135716, 'Streets_Slope_Mean': 0.9660569752256075, 'MultiUseTrails_Slope_Mean': None, 'trails_Slope_Mean': None, 'Grid_Slope_MEAN': 2.2087172894842113}
IndexID: 3, Data: {'Sidewalks_Slope_Mean': None, 'Streets_Slope_Mean': 3.095643554415022, 'MultiUseTrails_Slope_Mean': None, 'trails_Slope_Mean': 2.7575330917651844, 'Grid_Slope_MEAN': 2.269062724378373}
IndexID: 4, Data: {'S

#### Business Density Processing Loop

In [17]:
# List all business amenity point layers to apply kernel density
business_layers = [layer_name for layer_name in base_layers if layer_name.endswith("_Amenities")]

# Define the path to the neighborhood layer that will be used as a barrier
barrier_layer = "neighborhoods"

for layer_name in business_layers:
    print(f"Processing Kernel Density for business layer: {layer_name}")

    input_layer = f"{base_layers_group}\\{layer_name}"

    if not arcpy.Exists(input_layer):
        print(f"Layer {input_layer} does not exist. Skipping.")
        continue

    # Step 1: Clip the Point Features by the Barrier Layer
    clipped_points = f"{workspace}\\{layer_name}_clipped"
    arcpy.analysis.Clip(
        in_features=input_layer,
        clip_features=barrier_layer,
        out_feature_class=clipped_points
    )
    print(f"Clipped points for {layer_name} using the barrier layer.")

    # Step 2: Apply Kernel Density Tool to Generate Business Density Raster
    kernel_density_output = f"{workspace}\\{layer_name}_kernel_density"
    arcpy.sa.KernelDensity(
        in_features=clipped_points,
        population_field="NONE",
        out_cell_values="DENSITIES",
        method='GEODESIC',
        cell_size="6.59609600103067E-04",  # Match cell size to your fishnet grid resolution
        search_radius="200",  # Adjust based on the intended influence
        area_unit_scale_factor="SQUARE_FEET"
    ).save(kernel_density_output)
    print(f"Generated kernel density heatmap for {layer_name} using clipped features.")

    # Step 3: Apply Smoothing Using Focal Statistics
    smoothed_kernel_density_output = f"{workspace}\\{layer_name}_smoothed_kernel_density"
    smoothed_raster = arcpy.sa.FocalStatistics(
        in_raster=kernel_density_output,
        neighborhood=arcpy.sa.NbrCircle(radius=8, units="CELL"),
        statistics_type="MEAN"
    )
    smoothed_raster.save(smoothed_kernel_density_output)
    print(f"Applied focal statistics to smooth the kernel density heatmap for {layer_name}.")

    # Step 4: Calculate Average Density for Each Fishnet Grid Using Zonal Statistics
    zonal_output_table = f"{workspace}\\{layer_name}_zonal_stats"
    calculate_average_density(walkscore_fishnet_layer, smoothed_kernel_density_output, walkscore_fishnet_layer, zonal_output_table)

    # Step 5: Join Zonal Statistics Back to Fishnet Grid
    business_density_field = "business_density"

    # Delete the existing "business_density" field if it already exists
    if any(f.name == business_density_field for f in arcpy.ListFields(walkscore_fishnet_layer)):
        arcpy.management.DeleteField(walkscore_fishnet_layer, business_density_field)
        print(f"Deleted existing field '{business_density_field}'.")

    # Add the new "business_density" field
    arcpy.management.AddField(walkscore_fishnet_layer, business_density_field, "DOUBLE")
    print(f"Added new field '{business_density_field}'.")

    # Join the mean values from the zonal stats output back to the fishnet
    arcpy.management.JoinField(
        in_data=walkscore_fishnet_layer,
        in_field="IndexID",
        join_table=zonal_output_table,
        join_field="IndexID",
        fields=["MEAN"]
    )

    # Update the "business_density" field using the "MEAN" field from the joined table, setting nulls to 0
    with arcpy.da.UpdateCursor(walkscore_fishnet_layer, ["MEAN", business_density_field]) as cursor:
        for row in cursor:
            row[1] = row[0] if row[0] is not None else 0  # Copy the value from the "MEAN" field or set to 0 if None
            cursor.updateRow(row)

    # Delete the "MEAN" field after copying values
    arcpy.management.DeleteField(walkscore_fishnet_layer, "MEAN")
    print(f"Deleted temporary field 'MEAN' after updating 'business_density'.")

    print(f"Updated 'business_density' field with mean density values for {layer_name}.")

print("Business density processing complete for all layers.")

Processing Kernel Density for business layer: Business_Amenities
Clipped points for Business_Amenities using the barrier layer.
Generated kernel density heatmap for Business_Amenities using clipped features.
Applied focal statistics to smooth the kernel density heatmap for Business_Amenities.
Average density calculated and saved to C:\Users\rtvpd\Documents\Walkability_Seattle\Walkability_Seattle.gdb\Business_Amenities_zonal_stats.
Added new field 'business_density'.
Deleted temporary field 'MEAN' after updating 'business_density'.
Updated 'business_density' field with mean density values for Business_Amenities.
Business density processing complete for all layers.


In [18]:
# Step 4: Rank Normalize the Business Density Field Between 0-5
business_density_values = []

with arcpy.da.SearchCursor(walkscore_fishnet_layer, ["business_density"]) as cursor:
    for row in cursor:
        if row[0] is not None:
            business_density_values.append(row[0])

# Step 4.2: Calculate min and max values of business density
min_value = min(business_density_values) if business_density_values else 0
max_value = max(business_density_values) if business_density_values else 1  # Avoid division by zero

# Step 4.3: Update the "business_density" field using min-max normalization between 0-5
with arcpy.da.UpdateCursor(walkscore_fishnet_layer, ["business_density"]) as cursor:
    for row in cursor:
        if row[0] is not None:
            # Apply min-max normalization to scale between 0-5
            if max_value > min_value:
                row[0] = ((row[0] - min_value) / (max_value - min_value)) * 5
            else:
                row[0] = 0  # In case all values are the same, set normalized value to 0
        else:
            row[0] = 0  # Set to 0 for null business density
        cursor.updateRow(row)

print("Min-max normalized 'business_density' field to a scale of 0-5.")

Min-max normalized 'business_density' field to a scale of 0-5.


#### Generating an Index Field for walkscore_fishnet

In [19]:
# Populate the new index field with unique values
with arcpy.da.UpdateCursor(walkscore_fishnet_layer, [index_field]) as cursor:
    for i, row in enumerate(cursor):
        row[0] = i + 1
        cursor.updateRow(row)

### Step 3:  Join Summary Statistic Tables

In [20]:
merged_summary = f"{output_gdb}\\merged_sums"

if arcpy.Exists(merged_summary):
    arcpy.management.Delete(merged_summary)
    print('deleted existing summary table')
    
arcpy.management.CreateTable(output_gdb, "merged_sums")
print("Created merged_sums table.")

deleted existing summary table
Created merged_sums table.


In [21]:
# Add IndexID field to the merged summary table if it doesn't exist
if not any(f.name == index_field for f in arcpy.ListFields(merged_summary)):
    arcpy.management.AddField(merged_summary, index_field, "LONG")

# Create a dictionary to store the aggregated sums
aggregated_sums = defaultdict(lambda: defaultdict(float))

# Iterate through each summary table and aggregate values by IndexID
for layer_name in base_layers:
    summary_output = f"{output_gdb}\\{layer_name}_sum"
    
    # Verify if summary_output exists
    if not arcpy.Exists(summary_output):
        print(f"Summary table {summary_output} does not exist. Skipping.")
        continue

    fields = arcpy.ListFields(summary_output)
    field_names = [f.name for f in fields if f.name != index_field]
    
    # Aggregate the summary fields into the dictionary
    with arcpy.da.SearchCursor(summary_output, [index_field] + field_names) as cursor:
        for row in cursor:
            idx = row[0]
            for i, field_name in enumerate(field_names):
                value = row[i+1] if row[i+1] is not None else 0
                aggregated_sums[idx][f"{layer_name}_{field_name}"] += value

# Add aggregated fields to the merged summary table
for layer_name in base_layers:
    summary_output = f"{output_gdb}\\{layer_name}_sum"
    
    # Verify if summary_output exists
    if not arcpy.Exists(summary_output):
        print(f"Summary table {summary_output} does not exist. Skipping.")
        continue

    fields = arcpy.ListFields(summary_output)
    for field in fields:
        if field.name != index_field:
            field_name = f"{layer_name}_{field.name}"
            if not any(f.name == field_name for f in arcpy.ListFields(merged_summary)):
                arcpy.management.AddField(merged_summary, field_name, "DOUBLE")

# Insert the aggregated sums into the merged summary table
field_names_to_insert = [index_field] + [f"{layer_name}_{field.name}" for layer_name in base_layers for field in arcpy.ListFields(f"{output_gdb}\\{layer_name}_sum") if field.name != index_field]
with arcpy.da.InsertCursor(merged_summary, field_names_to_insert) as cursor:
    for idx, fields in aggregated_sums.items():
        row = [idx] + [fields.get(field_name, 0) for field_name in field_names_to_insert if field_name != index_field]
        cursor.insertRow(row)

print("Aggregated merged summary table created successfully.")

Aggregated merged summary table created successfully.


In [22]:
# Path to the merged summary table
merged_summary = r"C:\Users\rtvpd\Documents\Walkability_Seattle\Walkability_Seattle.gdb\merged_sums"
index_field = "IndexID"

# Check if merged summary table exists
if not arcpy.Exists(merged_summary):
    raise ValueError(f"{merged_summary} does not exist.")

# List all fields in the merged summary table for debugging
fields = arcpy.ListFields(merged_summary)
field_names = [field.name for field in fields]
print("Fields in merged summary table:", field_names)

# Ensure IndexID exists in merged_sums
if not any(f.name == index_field for f in fields):
    print(f"Adding {index_field} to {merged_summary}.")
    arcpy.management.AddField(merged_summary, index_field, "LONG")
else:
    print(f"{index_field} already exists in {merged_summary}.")

print("Verified IndexID in merged_sums.")

Fields in merged summary table: ['OBJECTID', 'IndexID', 'Business_Amenities_OBJECTID', 'Business_Amenities_FREQUENCY', 'Business_Amenities_COUNT_osm_business_id', 'Industrial_OBJECTID', 'Industrial_FREQUENCY', 'Industrial_SUM_Industrial_effective_area', 'ParkingLots_OBJECTID', 'ParkingLots_FREQUENCY', 'ParkingLots_SUM_ParkingLots_effective_area', 'GolfCourse_OBJECTID', 'GolfCourse_FREQUENCY', 'GolfCourse_SUM_GolfCourse_effective_area', 'Cemeteries_OBJECTID', 'Cemeteries_FREQUENCY', 'Cemeteries_SUM_Cemeteries_effective_area', 'Hospitals_OBJECTID', 'Hospitals_FREQUENCY', 'Hospitals_SUM_Hospitals_effective_area', 'Slope_OBJECTID', 'Slope_COUNT', 'Slope_AREA', 'Slope_MEAN', 'Bike_greenways_OBJECTID', 'Bike_greenways_FREQUENCY', 'Bike_greenways_SUM_Bike_greenways_area', 'Bike_protected_OBJECTID', 'Bike_protected_FREQUENCY', 'Bike_protected_SUM_Bike_protected_area', 'Bike_buffer_OBJECTID', 'Bike_buffer_FREQUENCY', 'Bike_buffer_SUM_Bike_buffer_area', 'Healthy_Streets_OBJECTID', 'Healthy_Stree

### Step 4: Join the Summary Statistics to the Fishnet Layer

In [23]:
try:
    arcpy.management.JoinField(walkscore_fishnet_layer, "IndexID", merged_summary, "IndexID")
except Exception as e:
    print(f"Error during join: {e}")

#### Adding a neighborhood field to Walkscore Fishnet

In [24]:
# Set environment settings
arcpy.env.overwriteOutput = True  # Allow outputs to be overwritten

# Define the input layers
walkscore_fishnet = r"C:\Users\rtvpd\Documents\Walkability_Seattle\Walkability_Seattle.gdb\walkscore_fishnet"
neighborhoods = r"C:\Users\rtvpd\Documents\Walkability_Seattle\Walkability_Seattle.gdb\neighborhoods"
fishnet_neighborhoods_intersect = "fishnet_neighborhoods_intersect"
neighborhood_field = "nested"  # Field name to be used from neighborhoods layer

# Step 1: Ensure the spatial reference systems match
walkscore_sr = arcpy.Describe(walkscore_fishnet).spatialReference
neighborhoods_sr = arcpy.Describe(neighborhoods).spatialReference

if walkscore_sr.name != neighborhoods_sr.name:
    raise ValueError("Spatial references do not match between walkscore_fishnet and neighborhoods.")

# Step 2: Intersect fishnet with neighborhoods to split grids at boundaries
arcpy.analysis.Intersect([walkscore_fishnet, neighborhoods], fishnet_neighborhoods_intersect)
print("Intersected walkscore fishnet with neighborhoods to create fragments.")

# Step 3: Verify that the 'nested', 'is_tourist', and 'is_industrial' fields are present in the intersected layer
fields = arcpy.ListFields(fishnet_neighborhoods_intersect)
field_names = [field.name for field in fields]

required_fields = [neighborhood_field, "is_tourist", "is_industrial"]
for field in required_fields:
    if field not in field_names:
        raise ValueError(f"Field '{field}' not found in the intersected layer.")

print(f"Using fields: {required_fields}")

# Optional: Calculate the area of each fragment for further analysis
arcpy.management.AddField(fishnet_neighborhoods_intersect, "Fragment_Area", "DOUBLE")
arcpy.management.CalculateGeometryAttributes(fishnet_neighborhoods_intersect, [["Fragment_Area", "AREA_GEODESIC"]])
print("Calculated area for each fragment.")

# Step 4: Use the intersected result directly as the new walkscore_fishnet
# Rename the intersected layer to replace the original walkscore_fishnet
arcpy.management.Delete(walkscore_fishnet)  # Delete the original fishnet to allow overwriting
arcpy.management.Rename(fishnet_neighborhoods_intersect, walkscore_fishnet)
print(f"Overwritten {walkscore_fishnet} with the intersected fragments, retaining neighborhood names.")

# Verify the output
print(f"Fields in {walkscore_fishnet} after processing:")
fields = arcpy.ListFields(walkscore_fishnet)
for field in fields:
    print(f"Name: {field.name}, Type: {field.type}")

print("Fishnet fragments assigned to neighborhoods and saved back to walkscore_fishnet.")

Intersected walkscore fishnet with neighborhoods to create fragments.
Using fields: ['nested', 'is_tourist', 'is_industrial']
Calculated area for each fragment.
Overwritten C:\Users\rtvpd\Documents\Walkability_Seattle\Walkability_Seattle.gdb\walkscore_fishnet with the intersected fragments, retaining neighborhood names.
Fields in C:\Users\rtvpd\Documents\Walkability_Seattle\Walkability_Seattle.gdb\walkscore_fishnet after processing:
Name: OBJECTID, Type: OID
Name: Shape, Type: Geometry
Name: FID_walkscore_fishnet, Type: Integer
Name: IndexID, Type: Integer
Name: total_area, Type: Double
Name: Max_Speed_Limit, Type: Double
Name: SUM_proportional_population, Type: Double
Name: COUNT_OBJECTID, Type: Integer
Name: Grid_Slope_MEAN, Type: Double
Name: Sidewalks_Slope_Mean, Type: Double
Name: Streets_Slope_Mean, Type: Double
Name: MultiUseTrails_Slope_Mean, Type: Double
Name: trails_Slope_Mean, Type: Double
Name: effective_slope, Type: Double
Name: business_density, Type: Double
Name: IndexID

#### Scaling MAX Speed Limit for Different Uses

In [25]:
speed_limit_scalers = {
    "Industrial": 1.25,
    "ParkingLots": 1.1,
    "GolfCourse": 1.1,
    "Cemeteries": 1.1,
    "Hospitals": 1.5,
#     "MultiUseTrails": 0.9,
#     "Parks": 0.5
}

In [26]:
# Add binary fields to walkscore_fishnet_layer for each area type
for area_type in speed_limit_scalers.keys():
    binary_field = f"Is{area_type.replace(' ', '')}"
    if not any(f.name == binary_field for f in arcpy.ListFields(walkscore_fishnet_layer)):
        arcpy.management.AddField(walkscore_fishnet_layer, binary_field, "SHORT")

In [27]:
# Populate binary fields based on effective area presence
for area_type in speed_limit_scalers.keys():
    effective_area_field = f"{area_type}_SUM_{area_type.replace(' ', '')}_effective_area"
    binary_field = f"Is{area_type.replace(' ', '')}"
    
    if any(f.name == effective_area_field for f in arcpy.ListFields(walkscore_fishnet_layer)):
        with arcpy.da.UpdateCursor(walkscore_fishnet_layer, [effective_area_field, binary_field]) as cursor:
            for row in cursor:
                effective_area = row[0] if row[0] is not None else 0
                row[1] = 1 if effective_area > 0 else 0
                cursor.updateRow(row)

print("Binary fields updated based on effective area presence.")

Binary fields updated based on effective area presence.


In [28]:
# Set the maximum and minimum allowable scaler values
max_scaler_value = speed_limit_scalers["Hospitals"]  # Maximum allowable scaler value
min_scaler_value = 1    # Minimum allowable scaler value

# Adjust Max_Speed_Limit using the binary fields and scalers with a cap on the scaling
with arcpy.da.UpdateCursor(walkscore_fishnet_layer, ["Max_Speed_Limit"] + [f"Is{area_type.replace(' ', '')}" for area_type in speed_limit_scalers.keys()]) as cursor:
    for row in cursor:
        max_speed_limit = row[0]
        applied_scaler = 1.0
        
        # Apply scalers based on binary fields
        if max_speed_limit is not None:
            for i, area_type in enumerate(speed_limit_scalers.keys(), start=1):
                if row[i] == 1:  # If binary field is 1, apply the scaler
                    applied_scaler *= speed_limit_scalers[area_type]

                    # Ensure applied_scaler does not exceed the max_scaler_value
                    if applied_scaler > max_scaler_value:
                        applied_scaler = max_scaler_value
                        break  # No need to continue if we've hit the max scaler

            # Ensure applied_scaler does not fall below the min_scaler_value
            if applied_scaler < min_scaler_value:
                applied_scaler = min_scaler_value

            # Apply the final scaler to the max speed limit
            max_speed_limit *= applied_scaler
            row[0] = max_speed_limit
            cursor.updateRow(row)

print("Max_Speed_Limit adjusted based on area type scalers with a ceiling and floor on scaling.")

Max_Speed_Limit adjusted based on area type scalers with a ceiling and floor on scaling.


### Step 5: Finalizing Walkscore Fishnet

Finally, we'll take the fishnet (walkscore_fishnet) and trim the fields down to only the mandatory fields (and permanent fields). This will include the calculation of the amenity density, which allows me to remove the count fields before passing the fishnet to the next notebook for walkscore calculation.

In [29]:
walkscore_fishnet = f"{output_gdb}\\walkscore_fishnet"

In [30]:
arcpy.env.overwriteOutput = True  # Allow outputs to be overwritten

# Define the input layers
neighborhoods_layer = r"C:\Users\rtvpd\Documents\Walkability_Seattle\Walkability_Seattle.gdb\neighborhoods"
fishnet_layer = r"C:\Users\rtvpd\Documents\Walkability_Seattle\Walkability_Seattle.gdb\walkscore_fishnet"
tree_canopy_layer = r"C:\Users\rtvpd\Documents\Walkability_Seattle\Walkability_Seattle.gdb\TreeCanopy"

In [31]:
def rank_normalize(value, values):
    sorted_values = sorted(values)
    rank = sorted_values.index(value) + 1
    return rank / len(values) * 10

#### Calculation of Crime Density Using Kernel Density

In [32]:
# Define the path to the neighborhood layer that will be used as a barrier
barrier_layer = "citylimits"

# Input layer for crime points (e.g., "crime_data_points")
crime_layer_name = "SPD_Crime_Data"
input_layer = f"{base_layers_group}\\{crime_layer_name}"

# Check if the input layer exists
if not arcpy.Exists(input_layer):
    print(f"Layer {input_layer} does not exist. Skipping.")
else:
    # Step 1: Clip the Crime Points by the Barrier Layer
    clipped_crime_points = f"{workspace}\\{crime_layer_name}_clipped"
    arcpy.analysis.Clip(
        in_features=input_layer,
        clip_features=barrier_layer,
        out_feature_class=clipped_crime_points
    )
    print(f"Clipped points for {crime_layer_name} using the barrier layer.")

    # Step 2: Apply Kernel Density Tool to Generate Crime Density Raster
    kernel_density_output = f"{workspace}\\{crime_layer_name}_kernel_density"
    arcpy.sa.KernelDensity(
        in_features=clipped_crime_points,
        population_field="NONE",  # Assuming the point occurrences are counts, so no separate population field
        out_cell_values="DENSITIES",
        method='GEODESIC',
        cell_size="7.45462399999951E-04",  # Match cell size to your fishnet grid resolution
        search_radius="500",  # Adjust based on the intended influence
        area_unit_scale_factor="SQUARE_FEET"
    ).save(kernel_density_output)
    print(f"Generated kernel density heatmap for {crime_layer_name} using clipped features.")

    # Step 3: Apply Smoothing Using Focal Statistics (optional)
    smoothed_kernel_density_output = f"{workspace}\\smoothed_crime_kernel_density"
    smoothed_crime_raster = arcpy.sa.FocalStatistics(
        in_raster=kernel_density_output,
        neighborhood=arcpy.sa.NbrCircle(radius=3, units="CELL"),  # Circular neighborhood with radius of 3 cells
        statistics_type="MEAN"
    )
    smoothed_crime_raster.save(smoothed_kernel_density_output)
    print(f"Applied focal statistics to smooth the kernel density heatmap for {crime_layer_name}.")

    # Step 4: Calculate Average Crime Density for Each Fishnet Grid Using Zonal Statistics
    zonal_output_table = f"{workspace}\\{crime_layer_name}_zonal_stats"
    calculate_average_density(walkscore_fishnet_layer, smoothed_kernel_density_output, walkscore_fishnet_layer, zonal_output_table)

    # Step 5: Join Zonal Statistics Back to Fishnet Grid
    crime_density_field = "crime_density"

    # Delete the existing "crime_density" field if it already exists
    if any(f.name == crime_density_field for f in arcpy.ListFields(walkscore_fishnet_layer)):
        arcpy.management.DeleteField(walkscore_fishnet_layer, crime_density_field)
        print(f"Deleted existing field '{crime_density_field}'.")

    # Add the new "crime_density" field
    arcpy.management.AddField(walkscore_fishnet_layer, crime_density_field, "DOUBLE")
    print(f"Added new field '{crime_density_field}'.")

    # Join the mean values from the zonal stats output back to the fishnet
    arcpy.management.JoinField(
        in_data=walkscore_fishnet_layer,
        in_field="IndexID",
        join_table=zonal_output_table,
        join_field="IndexID",
        fields=["MEAN"]
    )

    # Update the "crime_density" field using the "MEAN" field from the joined table, setting nulls to 0
    with arcpy.da.UpdateCursor(walkscore_fishnet_layer, ["MEAN", crime_density_field]) as cursor:
        for row in cursor:
            row[1] = row[0] if row[0] is not None else 0  # Copy the value from the "MEAN" field or set to 0 if None
            cursor.updateRow(row)

    # Delete the "MEAN" field after copying values
    arcpy.management.DeleteField(walkscore_fishnet_layer, "MEAN")
    print(f"Deleted temporary field 'MEAN' after updating 'crime_density'.")

    print(f"Updated 'crime_density' field with mean density values for {crime_layer_name}.")

print("Crime density processing complete.")

Clipped points for SPD_Crime_Data using the barrier layer.
Generated kernel density heatmap for SPD_Crime_Data using clipped features.
Applied focal statistics to smooth the kernel density heatmap for SPD_Crime_Data.
Average density calculated and saved to C:\Users\rtvpd\Documents\Walkability_Seattle\Walkability_Seattle.gdb\SPD_Crime_Data_zonal_stats.
Added new field 'crime_density'.
Deleted temporary field 'MEAN' after updating 'crime_density'.
Updated 'crime_density' field with mean density values for SPD_Crime_Data.
Crime density processing complete.


In [33]:
# Simple Normalization of Crime Density to a Range of 0-5
crime_density_field = "crime_density"
normalized_field = "crime_density_normalized"

# Ensure the normalized field does not already exist
if not any(f.name == normalized_field for f in arcpy.ListFields(walkscore_fishnet_layer)):
    arcpy.management.AddField(walkscore_fishnet_layer, normalized_field, "DOUBLE")
    print(f"Added '{normalized_field}' field to '{walkscore_fishnet_layer}'.")

# Step 1: Fetch all crime density values to calculate min and max
crime_density_values = []
with arcpy.da.SearchCursor(walkscore_fishnet_layer, [crime_density_field]) as cursor:
    for row in cursor:
        if row[0] is not None:
            crime_density_values.append(row[0])

# Calculate the min and max values
min_value = min(crime_density_values) if crime_density_values else 0
max_value = max(crime_density_values) if crime_density_values else 1  # Avoid division by zero

# Step 2: Update the normalized field in the fishnet grid
with arcpy.da.UpdateCursor(walkscore_fishnet_layer, [crime_density_field, normalized_field]) as cursor:
    for row in cursor:
        if row[0] is not None:
            # Apply min-max normalization to scale between 0-5
            if max_value > min_value:
                row[1] = ((row[0] - min_value) / (max_value - min_value)) * 5
            else:
                row[1] = 0  # In case all values are the same, set normalized value to 0
        else:
            row[1] = 0  # Set to 0 for null or zero crime density
        cursor.updateRow(row)

print(f"Normalized '{crime_density_field}' and updated '{normalized_field}' on a scale of 0-5 for each fishnet grid.")

Added 'crime_density_normalized' field to 'C:\Users\rtvpd\Documents\Walkability_Seattle\Walkability_Seattle.gdb\walkscore_fishnet'.
Normalized 'crime_density' and updated 'crime_density_normalized' on a scale of 0-5 for each fishnet grid.


#### Crash Density Calculation for Entire Fishnet Grid using Kernel Density

In [34]:
# Define the path to the neighborhood layer that will be used as a barrier
barrier_layer = "citylimits"

# Input layer for crash points (e.g., "crash_data_points")
crash_layer_name = "crashes"
input_layer = f"{base_layers_group}\\{crash_layer_name}"

# Check if the input layer exists
if not arcpy.Exists(input_layer):
    print(f"Layer {input_layer} does not exist. Skipping.")
else:
    # Step 1: Clip the Crash Points by the Barrier Layer
    clipped_crash_points = f"{workspace}\\{crash_layer_name}_clipped"
    arcpy.analysis.Clip(
        in_features=input_layer,
        clip_features=barrier_layer,
        out_feature_class=clipped_crash_points
    )
    print(f"Clipped points for {crash_layer_name} using the barrier layer.")

    # Step 2: Apply Kernel Density Tool to Generate Crash Density Raster
    kernel_density_output = f"{workspace}\\{crash_layer_name}_kernel_density"
    arcpy.sa.KernelDensity(
        in_features=clipped_crash_points,
        population_field="FATALITIES",
        out_cell_values="DENSITIES",
        method='GEODESIC',
        cell_size="7.45462399999951E-04",  # Match cell size to your fishnet grid resolution
        search_radius="500",  # Adjust based on the intended influence
        area_unit_scale_factor="SQUARE_FEET"
    ).save(kernel_density_output)
    print(f"Generated kernel density heatmap for {crash_layer_name} using clipped features.")

    # Step 3: Apply Smoothing Using Focal Statistics (optional)
    smoothed_kernel_density_output = f"{workspace}\\smoothed_crash_kernel_density"
    smoothed_crash_raster = arcpy.sa.FocalStatistics(
        in_raster=kernel_density_output,
        neighborhood=arcpy.sa.NbrCircle(radius=3, units="CELL"),  # Circular neighborhood with radius of 3 cells
        statistics_type="MEAN"
    )
    smoothed_crash_raster.save(smoothed_kernel_density_output)
    print(f"Applied focal statistics to smooth the kernel density heatmap for {crash_layer_name}.")

    # Step 4: Calculate Average Crash Density for Each Fishnet Grid Using Zonal Statistics
    zonal_output_table = f"{workspace}\\{crash_layer_name}_zonal_stats"
    calculate_average_density(walkscore_fishnet_layer, smoothed_kernel_density_output, walkscore_fishnet_layer, zonal_output_table)

    # Step 5: Join Zonal Statistics Back to Fishnet Grid
    crash_density_field = "crash_density"

    # Delete the existing "crash_density" field if it already exists
    if any(f.name == crash_density_field for f in arcpy.ListFields(walkscore_fishnet_layer)):
        arcpy.management.DeleteField(walkscore_fishnet_layer, crash_density_field)
        print(f"Deleted existing field '{crash_density_field}'.")

    # Add the new "crash_density" field
    arcpy.management.AddField(walkscore_fishnet_layer, crash_density_field, "DOUBLE")
    print(f"Added new field '{crash_density_field}'.")

    # Join the mean values from the zonal stats output back to the fishnet
    arcpy.management.JoinField(
        in_data=walkscore_fishnet_layer,
        in_field="IndexID",
        join_table=zonal_output_table,
        join_field="IndexID",
        fields=["MEAN"]
    )

    # Update the "crash_density" field using the "MEAN" field from the joined table, setting nulls to 0
    with arcpy.da.UpdateCursor(walkscore_fishnet_layer, ["MEAN", crash_density_field]) as cursor:
        for row in cursor:
            row[1] = row[0] if row[0] is not None else 0  # Copy the value from the "MEAN" field or set to 0 if None
            cursor.updateRow(row)

    # Delete the "MEAN" field after copying values
    arcpy.management.DeleteField(walkscore_fishnet_layer, "MEAN")
    print(f"Deleted temporary field 'MEAN' after updating 'crash_density'.")

    print(f"Updated 'crash_density' field with mean density values for {crash_layer_name}.")

print("Crash density processing complete.")

Clipped points for crashes using the barrier layer.
Generated kernel density heatmap for crashes using clipped features.
Applied focal statistics to smooth the kernel density heatmap for crashes.
Average density calculated and saved to C:\Users\rtvpd\Documents\Walkability_Seattle\Walkability_Seattle.gdb\crashes_zonal_stats.
Added new field 'crash_density'.
Deleted temporary field 'MEAN' after updating 'crash_density'.
Updated 'crash_density' field with mean density values for crashes.
Crash density processing complete.


In [35]:
# Simple Normalization of Crash Density to a Range of 0-5
crash_density_field = "crash_density"
normalized_field = "crash_density_normalized"

# Ensure the normalized field does not already exist
if not any(f.name == normalized_field for f in arcpy.ListFields(walkscore_fishnet_layer)):
    arcpy.management.AddField(walkscore_fishnet_layer, normalized_field, "DOUBLE")
    print(f"Added '{normalized_field}' field to '{walkscore_fishnet_layer}'.")

# Step 6: Fetch all crash density values to calculate min and max
crash_density_values = []
with arcpy.da.SearchCursor(walkscore_fishnet_layer, [crash_density_field]) as cursor:
    for row in cursor:
        if row[0] is not None:
            crash_density_values.append(row[0])

# Calculate the min and max values
min_value = min(crash_density_values) if crash_density_values else 0
max_value = max(crash_density_values) if crash_density_values else 1  # Avoid division by zero

# Step 7: Update the normalized field in the fishnet grid
with arcpy.da.UpdateCursor(walkscore_fishnet_layer, [crash_density_field, normalized_field]) as cursor:
    for row in cursor:
        if row[0] is not None:
            # Apply min-max normalization to scale between 0-5
            if max_value > min_value:
                row[1] = ((row[0] - min_value) / (max_value - min_value)) * 5
            else:
                row[1] = 0  # In case all values are the same, set normalized value to 0
        else:
            row[1] = 0  # Set to 0 for null or zero crash density
        cursor.updateRow(row)

print(f"Normalized '{crash_density_field}' and updated '{normalized_field}' on a scale of 0-5 for each fishnet grid.")

Added 'crash_density_normalized' field to 'C:\Users\rtvpd\Documents\Walkability_Seattle\Walkability_Seattle.gdb\walkscore_fishnet'.
Normalized 'crash_density' and updated 'crash_density_normalized' on a scale of 0-5 for each fishnet grid.


In [36]:
# Drop specified fields
fields_to_drop = []
for field in arcpy.ListFields(walkscore_fishnet):
    if field.name.endswith("FREQUENCY") or field.name.endswith("_OBJECTID") or field.name.endswith("Slope_AREA") or field.name.endswith("Slope_COUNT") or field.name.endswith("IndexID_1") or field.name.endswith("_id"):
        fields_to_drop.append(field.name)

if fields_to_drop:
    arcpy.management.DeleteField(walkscore_fishnet, fields_to_drop)

# Verify fields in walkscore_fishnet after dropping specified fields
walkscore_fishnet = f"{output_gdb}\\walkscore_fishnet"
print(f"\nFields in {walkscore_fishnet} after dropping specified fields:")
fields = arcpy.ListFields(walkscore_fishnet)
for field in fields:
    print(f"Name: {field.name}, Type: {field.type}")

print("Fields dropped successfully.")


Fields in C:\Users\rtvpd\Documents\Walkability_Seattle\Walkability_Seattle.gdb\walkscore_fishnet after dropping specified fields:
Name: OBJECTID, Type: OID
Name: Shape, Type: Geometry
Name: FID_walkscore_fishnet, Type: Integer
Name: IndexID, Type: Integer
Name: total_area, Type: Double
Name: Max_Speed_Limit, Type: Double
Name: SUM_proportional_population, Type: Double
Name: Grid_Slope_MEAN, Type: Double
Name: Sidewalks_Slope_Mean, Type: Double
Name: Streets_Slope_Mean, Type: Double
Name: MultiUseTrails_Slope_Mean, Type: Double
Name: trails_Slope_Mean, Type: Double
Name: effective_slope, Type: Double
Name: business_density, Type: Double
Name: Industrial_SUM_Industrial_effective_area, Type: Double
Name: ParkingLots_SUM_ParkingLots_effective_area, Type: Double
Name: GolfCourse_SUM_GolfCourse_effective_area, Type: Double
Name: Cemeteries_SUM_Cemeteries_effective_area, Type: Double
Name: Hospitals_SUM_Hospitals_effective_area, Type: Double
Name: Slope_MEAN, Type: Double
Name: Bike_greenway

### Step 6: Cleaning Contents Pane

In [37]:
def delete_created_layers(layers_list):
    for layer in layers_list:
        if arcpy.Exists(layer):
            arcpy.management.Delete(layer)
            print(f"Deleted layer: {layer}")

In [38]:
base_layers = [
    "Industrial",
    "ParkingLots",
    "GolfCourse",
    "Cemeteries",
    "Hospitals",
#     "Slope",
    "Bike_greenways",
    "Bike_protected",
    "Bike_buffer",
    "Healthy_Streets",
    "Parks",
    "Universities",
    "Sidewalks",
    "Plaza",
    "trails",
    "MultiUseTrails",
    "Streets",
    "fishnet_clipped",
    "Marked_Crosswalks",
    "fishnet_clipped",
    "neighborhoods",
    "population",
    "crashes"
]

In [39]:
base_layers_group = r"C:\Users\rtvpd\Documents\Walkability_Seattle\Walkability_Seattle.gdb"
target_spatial_reference = arcpy.SpatialReference(3857)  # WGS 1984 Web Mercator (auxiliary sphere)

def project_layer(input_layer, target_sr):
    input_layer_sr = arcpy.Describe(input_layer).spatialReference
    
    if input_layer_sr.name != target_sr.name:
        temp_projected_layer = os.path.join(arcpy.env.scratchGDB, f"{os.path.basename(input_layer)}_proj")
        arcpy.management.Project(input_layer, temp_projected_layer, target_sr)
        print(f"Projected {input_layer} to {temp_projected_layer}.")
        
        # Overwrite the original layer with the projected version
        arcpy.management.Delete(input_layer)
        arcpy.management.CopyFeatures(temp_projected_layer, input_layer)
        arcpy.management.Delete(temp_projected_layer)
        print(f"Replaced original {input_layer} with projected version.")
    else:
        print(f"{input_layer} is already in the target spatial reference.")

# Process each base layer
for layer_name in base_layers:
    print(f"Processing layer: {layer_name}")  # Debugging statement

    # Access the layer
    input_layer = f"{base_layers_group}\\{layer_name}"
    
    # Verify if the input_layer exists
    if not arcpy.Exists(input_layer):
        print(f"Layer {input_layer} does not exist. Skipping.")
        continue

    # Project the input layer to the target spatial reference
    project_layer(input_layer, target_spatial_reference)

print("All layers have been projected to the target spatial reference.")

Processing layer: Industrial
C:\Users\rtvpd\Documents\Walkability_Seattle\Walkability_Seattle.gdb\Industrial is already in the target spatial reference.
Processing layer: ParkingLots
C:\Users\rtvpd\Documents\Walkability_Seattle\Walkability_Seattle.gdb\ParkingLots is already in the target spatial reference.
Processing layer: GolfCourse
C:\Users\rtvpd\Documents\Walkability_Seattle\Walkability_Seattle.gdb\GolfCourse is already in the target spatial reference.
Processing layer: Cemeteries
C:\Users\rtvpd\Documents\Walkability_Seattle\Walkability_Seattle.gdb\Cemeteries is already in the target spatial reference.
Processing layer: Hospitals
C:\Users\rtvpd\Documents\Walkability_Seattle\Walkability_Seattle.gdb\Hospitals is already in the target spatial reference.
Processing layer: Bike_greenways
C:\Users\rtvpd\Documents\Walkability_Seattle\Walkability_Seattle.gdb\Bike_greenways is already in the target spatial reference.
Processing layer: Bike_protected
C:\Users\rtvpd\Documents\Walkability_Seat